# Introduction


This is a database of the water in the Chesapeake Bay. In the [Preparing to combine datasets section](#preparing-to-combine-datasets), run either [turn parameters into columns](#turn-parameters-into-columns) or [renaming columns instead of paramenters into columns](#renaming-columns-instead-of-paramenters-into-columns), as these are two different ways of preparting for the next step.

In [431]:
import pandas as pd
import numpy as np


The CSVs have the following columns. The individual CSVs will not have every column:


Here is a descriptor of the columns, from [The 2012 Users Guide to CBP Biological Monitoring Data](https://d18lev1ok5leia.cloudfront.net/chesapeakebay/documents/guide2012_final.pdf):
- `CBSeg2003` 2003 Chesapeake Bay Segment Designation. Divided into regions based on circulation and salinity properties. We used 8 from the Bay proper, 2 adjoining Bays, and 1 adjoining sound.
- `CBSeg2003Description` 2003 Chesapeake Bay Segment Designation Description in the format Location-Salinity. The locations are Chesapeake Bay, Eastern Bay, Mobjack Bay, and Tangier Sound. The salinity levels are tidal fresh (0.0 - 0.5 parts per thousand),
oligohaline (0.5 - 5.0 parts per thousand), mesohaline (5.0 - 18.0 parts per thousand), and polyhaline (greater than 18.0 parts per thousand). 
- `Station` the sampling station
- `Latitude` and  `Longitude`, the Latitude and Longitude for the sampling station
- `FieldActivityId` and `EventId` are Database Generated Event Identification Numbers. Theses values are distint-- `FieldActivityId` is adding from the monitoring event data. Sediment and have `EventId` as well.
- `SampleDate` Sampling date (MM/DD/YYYY). 
- `SampleTime` Sample Collection Time (HHMM)
- `Layer` Layer of Water Column in Which Sample Was Taken. However, this column is not consistently coded.
- `TotalDepth` is Total Station Depth (Meters)
- `Source` Data Collection Agency
- `SampleReplicate` This parameter combines the sample replicate number with a sample type
descriptor. 
     - S1, Sample 1. The vast majority of the data.
     - S2, Sample 2 
- `ReportingParameter` (sediment), `IBIParameter` (biomass) Sampling Parameter. In this dataset, all are COUNT,  the number of cells per liter
- `ReportingValue` (taxonomic), `ReportedValue` (sediment), or `IBIValue` (bio mass) the value of the parameter.
- `ReportingUnits` This parameter describes the units in which a substance is measured. 
- `ProjectIdentifier`
- `Units` units for the sample volume. Always centimeters (cubic centimeters?)
- `SampleVolume` Total Volume of Sample
- `PDepth`, Composite Sample Cut Off Depth (meters)
- `Salzone`, Salinity Zone
     - HM, High Mesohaline =>12 TO 18 parts per thousand
     - LM, Low Mesohaline =>5.0 TO 12 parts per thousand
     - M, Mesohaline =>5.0 TO 18 parts per thousand
     - O, Oligohaline =>0.5 TO 5.0 parts per thousand
     - P, Polyhaline =>18 parts per thousand
     - TF, Tidal Fresh < 0.5 parts per thousand

Additionally, the taxonomic dataset has 
- `GMethod` Chesapeake Bay Program Gear Method Code. Codes represent information relating to the type of field gear used to collect samples for all analysis. In this dataset all are 7, Plankton Pump
- `TSN` ITIS Taxon Serial Number, unique to the species. When used in conjunction with the NODC, the TSN
overcomes the problem of numeric changes in the NODC code whenever species are reclassified. 
- `LatinName` Species Latin Name 
- `Size` Cell Size Groupings when taken. Some species have different measurements for different sizes. 
- `LifeStageDescription`, a numeric code of the life stage. Most are 89 - not specified.

For initial cleaning, we will remove `Layer`, `Units`. Other cleaning will be done based on the dataset.

Then we will turn the paramenters into columns populated by the measured values.

Finally, combine all four datasets on the values for them monitoring data (other than layer and unit).

# Cleaning Individual Datasets

The first step of data cleaning is working with the individual datasets and determining what information we need to keep. Some of this cleaning will be done with the aid of the DataWrangler extension in VSCode. 

## Sediment

Opening `../data/plank_ChesapeakeBenthicSediment.csv` in DataWrangler shows that `SampleVolume` is missing 56 values. Let's replace '' and NaN with None and check again.

In [432]:
sediment = pd.read_csv('../data/plank_ChesapeakeBenthicSediment.csv')

sediment_clean = sediment.replace('', np.nan).where(sediment.notna(), None)

Drop `Layer` and `Units` columns

In [433]:
sediment_clean = sediment_clean.drop(columns=['Layer','Units'])

sediment_clean.shape

(4725, 19)

Use the `ReportingParameter` and `ReportingUnits` columns to create a dictionary, then remove the `ReportingUnits` column. We need to find the unique values in `ReportingParameter` to create a dictionary of their meanings. The information is in a PDF which cannot be scraped.

In [434]:
sediment_parameters = sediment_clean['ReportingParameter'].unique()

print(sediment_parameters)

['MOIST' 'SAND' 'TC' 'TIC' 'SILTCLAY' 'TOC' 'TN' 'VOLORG' 'KURTOSIS'
 'CLAY' 'MEANDIAM' 'SORT' 'SKEWNESS']


In [435]:
sediment_parameters_meanings = ['Sediment Moisture Percentage', 'Sand Content, Percent', 'Total Carbon Content','Total Inorganic Carbonate Content','Silt Clay Content, Percent','Total Organic Carbon','Total Nitrogen','Volatile Organic, Percent','Kurtosis','Clay Content, Percent','Mean Sediment Diameter','Sorting','Skewness']

In [436]:
# Create a mapping from parameters to their meanings
param_to_meaning = {param: meaning for param, meaning in zip(sediment_parameters, sediment_parameters_meanings)}

In [437]:
# Create the initial dictionary with units and empty types
sediment_param_dict = {param: {'Units': unit, 'Type': ""} for param, unit in zip(sediment_clean['ReportingParameter'], sediment_clean['ReportingUnits'])}

# Update the dictionary with the meanings
for param in sediment_param_dict:
    if param in param_to_meaning:
        sediment_param_dict[param]['Type'] = param_to_meaning[param]

In [438]:
sediment_clean = sediment_clean.drop(columns='ReportingUnits')

Let's rename `ReportingParameter` as `ReportedParameter`. This will allow us to define a tranformation function.

In [439]:
sediment_clean = sediment_clean.rename(columns={'ReportingParameter':'ReportedParameter'})

## BioMass

In [440]:
biomass = pd.read_csv('../data/plank_ChesapeakeBenthicBioMass.csv')

biomass_clean = biomass.replace('', np.nan).where(biomass.notna(), None)

Drop `Layer` and `Units` columns

In [441]:
biomass_clean = biomass_clean.drop(columns=['Layer','Units'])

biomass_clean.shape

(26904, 18)

Use the `IBIParameter` and `IBIValue` columns to create a dictionary, then remove the `IBIValue` column. We need to find the unique values in `IBIParameter` to create a dictionary of their meanings. The information is in a PDF which cannot be scraped, but ther are 126 unique values.

In [442]:
biomass_parameters = biomass_clean['IBIParameter'].unique()


There are also a few rows that do not encode information that were missed by the cleaning in the download. Let's drop the rows where `IBIParameter` is missing (from DataWrangler, these are the correct rows).

In [443]:
biomass_clean = biomass_clean.dropna(subset=['IBIParameter'])

Let's rename `IBIParameter` as `ReportedParameter` and `IBIValue` as `ReportedValue`. This will allow us to define a tranformation function.

In [444]:
biomass_clean = biomass_clean.rename(columns={'IBIParameter':'ReportedParameter','IBIValue':'ReportedValue'})

## Taxonomic Counts

Opening `../data/plank_ChesapeakeBenthicTaxonomic.csv` in DataWrangler shows there are also a few rows that do not encode information that were missed by the cleaning in the download. Let's drop the rows where `LatinName` is missing (from DataWrangler, these are the correct rows).

In [445]:
taxonomic = pd.read_csv('../data/plank_ChesapeakeBenthicTaxonomic.csv')

taxonomic_clean = taxonomic.replace('', np.nan).where(taxonomic.notna(), None)

taxonomic_clean = taxonomic_clean.dropna(subset=['LatinName'])

Drop `Layer` and `Units` columns

In [446]:
taxonomic_clean = taxonomic_clean.drop(columns=['Layer','Units'])

taxonomic_clean.shape

(26128, 22)

Let's create a dictionary for `LifeStageDescription`, from the table in the user guide.

In [447]:
taxonomic_clean['LifeStageDescription'].unique()

array([ 89.,  79., 248.,  76.,  97., 247., 245.,  53.,  21., 225.,  93.,
        52., 232.])

In [448]:
life_stage_dict = {
    89.: 'Not Specified',  
    79.: 'Species', 
    248.: 'Immature Without Cap. Chaete',  
    76.: 'Group',  
    97.: 'Larvae', 
    247.: 'Immature With Cap. Chaete', 
    245.: 'Type',  
    53.: 'Species B',  
    21.: 'Pupae', 
    225.: 'Complex',  
    93.: 'Juvenile',
    52.: 'Species A', 
    232.: 'Species M'
}

Since `LatinName` and `TSN` are already categorical data, we will still with the words version. Let's replace the `LifeStateDescription` with the actual description.

In [449]:
taxonomic_clean['LifeStageDescription'] = taxonomic_clean['LifeStageDescription'].replace(life_stage_dict)

Let's see how many `LatinNames` are measured at multiple life stages.

In [450]:
# Group by LatinName and check the unique values in LifeStageDescriptions
groups = taxonomic_clean.groupby('LatinName')['LifeStageDescription'].nunique()

# Identify LatinName values where LifeStageDescriptions has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
filtered_df = taxonomic_clean[taxonomic_clean['LatinName'].isin(disagreeing_a_values)]

filtered_df[['LatinName','LifeStageDescription']].drop_duplicates().sort_values(by='LatinName')

,LatinName,LifeStageDescription
24636,Ampelisca,Species
8691,Ampelisca,Juvenile
46,Axarus,Not Specified
641,Axarus,Species
11432,Bivalvia,Species
11076,Bivalvia,Species B
33,Chironomidae,Larvae
351,Chironomidae,Pupae
629,Chironomidae,Not Specified
25684,Enchytraeidae,Not Specified


We combine the `LatinName` and `LifeStageDescription` columns. We will drop `Not Specified`. It appears that in this dataset, any `LatinName` has at most one `LifeStageDescription` that contains `Species`, but we will keep those values to be safe, since some have 'Not Specified'.

In [451]:
taxonomic_clean['LatinName'] = taxonomic_clean['LatinName'] + ' ' +taxonomic_clean['LifeStageDescription'].replace('Not Specified', '')

taxonomic_clean= taxonomic_clean.drop(columns='LifeStageDescription')

Are `FieldActivityID` and `EventId` different numbering systems for the same thing?

In [452]:
# Group by FieldActivityId and check the unique values in EventId
groups = taxonomic_clean.groupby('FieldActivityId')['EventId'].nunique()

# Identify FieldActivityId values where EventId has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
filtered_df = taxonomic_clean[taxonomic_clean['FieldActivityId'].isin(disagreeing_a_values)]

filtered_df[['FieldActivityId','EventId']].drop_duplicates().sort_values(by='FieldActivityId')

,FieldActivityId,EventId


Yes, they are! Do they include depth? time? GMethod?

In [453]:
# Group by FieldActivityId and check the unique values in TotalDepth
groups = taxonomic_clean.groupby('FieldActivityId')['TotalDepth'].nunique()

# Identify FieldActivityId values where TotalDepth has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
filtered_df = taxonomic_clean[taxonomic_clean['FieldActivityId'].isin(disagreeing_a_values)]

print('TotalDepth',filtered_df[['FieldActivityId','TotalDepth']].drop_duplicates().sort_values(by='FieldActivityId').shape)

# Group by FieldActivityId and check the unique values in PDepth
groups = taxonomic_clean.groupby('FieldActivityId')['PDepth'].nunique()

# Identify FieldActivityId values where TotalDepth has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
filtered_df = taxonomic_clean[taxonomic_clean['FieldActivityId'].isin(disagreeing_a_values)]

print('PDepth',filtered_df[['FieldActivityId','PDepth']].drop_duplicates().sort_values(by='FieldActivityId').shape)

# Group by FieldActivityId and check the unique values in SampleTime
groups = taxonomic_clean.groupby('FieldActivityId')['SampleTime'].nunique()

# Identify FieldActivityId values where EventId has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
filtered_df = taxonomic_clean[taxonomic_clean['SampleTime'].isin(disagreeing_a_values)]

print('SampleTime',filtered_df[['FieldActivityId','SampleTime']].drop_duplicates().sort_values(by='SampleTime').shape)

# Group by FieldActivityId and check the unique values in GMethod
groups = taxonomic_clean.groupby('FieldActivityId')['GMethod'].nunique()

# Identify FieldActivityId values where EventId has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
filtered_df = taxonomic_clean[taxonomic_clean['GMethod'].isin(disagreeing_a_values)]

print('GMethod',filtered_df[['FieldActivityId','GMethod']].drop_duplicates().sort_values(by='GMethod').shape)

TotalDepth (0, 2)
PDepth (0, 2)
SampleTime (0, 2)
GMethod (0, 2)


Is `TSN` always the same for `LatinName`? 

In [454]:
# Group by LatinName and check the unique values in GMethod
groups = taxonomic_clean.groupby('LatinName')['TSN'].nunique()

# Identify LatinName values where EventId has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
taxonomic_clean[taxonomic_clean['TSN'].isin(disagreeing_a_values)].drop_duplicates

<bound method DataFrame.drop_duplicates of Empty DataFrame
Columns: [CBSeg2003, CBSeg2003Description, Station, Latitude, Longitude, SampleType, FieldActivityId, SampleDate, SampleTime, TotalDepth, ReportingValue, ReportingUnit, EventId, Source, GMethod, TSN, LatinName, ProjectIdentifier, Salzone, PDepth, SampleVolume]
Index: []

[0 rows x 21 columns]>

We will pivot the `LatinName` column and `ReportingValue` column. 
We drop `TSN` since we are using `LatinName` (the other option would be to also pivot on `TSN` and `ReportingValue`). Since `GMethod` is information about how the sample was collected, and no needed for merging datasets, we will also remove it. 

The `ReportingUnit` is always `CM`, so we can also drop it. Also, some of these measurements must be counts, unless some clams are over a kilometer in size.

In [455]:
taxonomic_clean = taxonomic_clean.drop(columns=['TSN','GMethod','ReportingUnit'])

### Dealing with ReportingValues column -- not used later

Since some values in the `ReportingValue` column appear to be counts and some are size, let's see if there are any rows that agree in all columns except `ReportingValue`

In [456]:
# Columns to exclude from comparison
exclude_columns = ['ReportingValue']

# Columns to compare
compare_columns = [col for col in taxonomic_clean.columns if col not in exclude_columns]

# Group by the columns to compare
grouped = taxonomic_clean.groupby(compare_columns)

# Initialize lists to collect rows based on group size
groups_with_2_rows = []
groups_with_3_rows = []
groups_with_more_than_3_rows = []

# Iterate through each group
for group_key, group in grouped:
    num_rows = len(group)
    
    # Separate groups based on number of rows
    if num_rows == 2:
        groups_with_2_rows.append(group)
    elif num_rows == 3:
        groups_with_3_rows.append(group)
    elif num_rows > 3:
        groups_with_more_than_3_rows.append(group)

# Create DataFrames for viewing
df_groups_with_2_rows = pd.concat(groups_with_2_rows).sort_values(by=['LatinName', 'EventId']) if groups_with_2_rows else pd.DataFrame()

df_groups_with_3_rows = pd.concat(groups_with_3_rows).sort_values(by=['LatinName', 'EventId','ReportingValue']) if groups_with_3_rows else pd.DataFrame()

df_groups_with_more_than_3_rows = pd.concat(groups_with_more_than_3_rows).sort_values(by=['LatinName', 'EventId','ReportingValue']).drop_duplicates() if groups_with_more_than_3_rows else pd.DataFrame()

I am not sure what to do about the groups with more than two rows, so let's continue with the groups with two rows and come back. For the groups with two rows, we will check if one of the values is an integer. If it is, then that will be the count and the other value will be the size.

In [457]:
# Initialize lists to accumulate results
results_list = []
error_groups_list = []

# Iterate over each group in the existing 'grouped' object
for group_key, group in grouped:
    if len(group) == 2:
        # Extract values from the group
        values = group['ReportingValue'].values
        
        # Check which values are integers
        int_values = [v for v in values if v == round(v)]
        
        if len(int_values) == 1:
            # One integer value found
            reporting_count = int_values[0]
            reporting_size = values[0] if values[0] != reporting_count else values[1]
            
            # Initialize new columns
            group = group.copy()
            group['ReportingCount'] = reporting_count
            group['ReportingSize'] = reporting_size
            
            # Append to results list
            results_list.append(group)
        else:
            # Append to error_groups list
            error_groups_list.append(group)

# Convert lists to DataFrames
taxonomic_two_measures = pd.concat(results_list, ignore_index=True) if results_list else pd.DataFrame(columns=taxonomic_clean.columns.tolist() + ['ReportingCount', 'ReportingSize'])

error_groups_df = pd.concat(error_groups_list, ignore_index=True) if error_groups_list else pd.DataFrame(columns=taxonomic_clean.columns)

# Sort the results DataFrame by 'LatinName' and 'EventId'
taxonomic_two_measures.sort_values(by=['LatinName', 'EventId'], inplace=True)

# Sort the error groups DataFrame by 'LatinName' and 'EventId'
error_groups_df.sort_values(by=['LatinName', 'EventId'], inplace=True)


The error group contains four rows where every value is an integer. We will handle that case and the groups of size 1 together. Let's generate the groups of size 1.

In [458]:
# Initialize list to accumulate single element groups
single_element_groups_list = []

# Iterate over each group in the existing 'grouped' object
for group_key, group in grouped:
    if len(group) == 1:
        # Append single element groups to single_element_groups_list
        single_element_groups_list.append(group)

# Convert the list to a DataFrame
taxonomic_one_measure = pd.concat(single_element_groups_list, ignore_index=True) if single_element_groups_list else pd.DataFrame(columns=taxonomic_clean.columns)

# Optionally, sort the DataFrame by 'LatinName'
taxonomic_one_measure.sort_values(by=['LatinName'], inplace=True)

 Since we will want to update this later, let's make a function.

In [459]:
def create_ranges_dict(dataframe, column_to_fix, ranges_columns):
    dictionary = {}
    for latin_name, group in dataframe.groupby(column_to_fix):
        col_dict = {}
        for col in ranges_columns:
            if col in group.columns:
                # Get the min and max values for each column
                reporting_range = [group[col].min(), group[col].max()]
                # Add the range to the column dictionary
                col_dict[col + 'Range'] = reporting_range
        # Store the column dictionary in the main dictionary
        dictionary[latin_name] = col_dict
    return dictionary

In [460]:
ranges_dict = create_ranges_dict(taxonomic_two_measures,'LatinName',['ReportingSize','ReportingCount'])

In [461]:
def check_value_in_ranges(row):
    latin_name = row['LatinName']
    reporting_value = row['ReportingValue']
    
    if latin_name in ranges_dict:
        count_range = ranges_dict[latin_name]['ReportingCountRange']
        size_range = ranges_dict[latin_name]['ReportingSizeRange']
        
        # Check if the reporting_value is within either range
        in_count_range = count_range[0] <= reporting_value <= count_range[1]
        in_size_range = size_range[0] <= reporting_value <= size_range[1]
        
        if in_count_range and in_size_range:
            return 'Both'
        elif in_count_range:
            # check if integer
            if reporting_value == round(reporting_value):
                return 'In Count Range'
            else:
                return 'In Count Range, not integer'
    
        elif in_size_range:
            return 'In Size Range'
        else:
            # check if integer
            if reporting_value == round(reporting_value):
                return 'Not in ranges, integer'
            else:
                return 'Not in ranges, not integer'
    else:
        # check if integer
        if reporting_value == round(reporting_value):
            return 'Taxa not in dictionary, integer'
        else:
            return 'Taxa not in dictionary, not integer'

Now we check if the `ReportedValue` is in the ranges.

In [462]:
taxonomic_one_measure['InRange'] = taxonomic_one_measure.apply(check_value_in_ranges, axis=1)

We can now add some of these values to the correct column to start creating our new dataset. We will remove those rows from `taxonomic_one_measure`

In [463]:
# Identify rows that can be updayed
condition = taxonomic_one_measure['InRange'].isin(['In Size Range', 'Not in ranges, not integer', 'Taxa not in dictionary, not integer']) | (taxonomic_one_measure['InRange'] == 'In Count Range')

# Create the new DataFrame with all columns and additional columns 'ReportingSize' and 'ReportingCount'
new_taxonomic_one_measure = taxonomic_one_measure[condition].copy()
new_taxonomic_one_measure['ReportingSize'] = new_taxonomic_one_measure.apply(
    lambda row: row['ReportingValue'] if row['InRange'] in ['In Size Range', 'Not in ranges, not integer', 'Taxa not in dictionary, not integer'] else None, 
    axis=1
)
new_taxonomic_one_measure['ReportingCount'] = new_taxonomic_one_measure.apply(
    lambda row: row['ReportingValue'] if row['InRange'] == 'In Count Range' else None, 
    axis=1
)

# Remove the identified rows from the original DataFrame while retaining all columns
# Drop InRanges to rerun
taxonomic_one_measure = taxonomic_one_measure.loc[~condition].drop(columns='InRange')

Now we do the same thing with the error group.

In [464]:
# add InRanges
error_groups_df['InRange'] = error_groups_df.apply(check_value_in_ranges, axis=1)

# Move the values we can
# Identify rows that can be updayed
condition = error_groups_df['InRange'].isin(['In Size Range', 'Not in ranges, not integer', 'Taxa not in dictionary, not integer']) | (error_groups_df['InRange'] == 'In Count Range')

# Create the new DataFrame with all columns and additional columns 'ReportingSize' and 'ReportingCount'
new_error_groups_df = error_groups_df[condition].copy()
new_error_groups_df['ReportingSize'] = new_error_groups_df.apply(
    lambda row: row['ReportingValue'] if row['InRange'] in ['In Size Range', 'Not in ranges, not integer', 'Taxa not in dictionary, not integer'] else None, 
    axis=1
)
new_error_groups_df['ReportingCount'] = new_error_groups_df.apply(
    lambda row: row['ReportingValue'] if row['InRange'] == 'In Count Range' else None, 
    axis=1
)

# Remove the identified rows from the original DataFrame while retaining all columns
# Drop InRanges to rerun
error_groups_df = error_groups_df.loc[~condition].drop(columns='InRange')

Now combine our dataframes, update the dictionary, and run ranges assessment again.

In [465]:
combined_taxonomic_data = pd.concat([new_error_groups_df,new_taxonomic_one_measure,taxonomic_two_measures],ignore_index=True).drop(columns=['ReportingValue','InRange']).drop_duplicates()


/var/folders/rl/kqt6tbv90l9_pwc4927vdb340000gn/T/ipykernel_30729/2626544837.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_taxonomic_data = pd.concat([new_error_groups_df,new_taxonomic_one_measure,taxonomic_two_measures],ignore_index=True).drop(columns=['ReportingValue','InRange']).drop_duplicates()


Recreate our dictionary and try again

## Water Quality

Opening `../data/plank_ChesapeakeBenthicWaterQuality.csv` in DataWrangler shows that `SampleVolume` is missing 94 values.

In [466]:
water = pd.read_csv('../data/plank_ChesapeakeBenthicWaterQuality.csv')

water_clean = water.replace('', np.nan).where(water.notna(), None)

Drop `Layer` and `Units` columns

In [467]:
water_clean = water_clean.drop(columns=['Layer','Units'])

water_clean.shape

(8427, 21)

Use the `ReportedParameter` and `ReportingUnits` columns to create a dictionary, then remove the `ReportingUnits` column. We need to find the unique values in `ReportingParameter` to create a dictionary of their meanings. The information is in a PDF which cannot be scraped.

In [468]:
water_parameters = water_clean['ReportedParameter'].unique()

print(water_parameters)

['PH' 'WTEMP' 'DO' 'DO_SAT_P' 'SALINITY' 'SPCOND']


Create a dictionary for the parameters. All are method F01, in-situ measurements.

In [469]:
# Create the initial dictionary with units and empty types
water_param_dict = {'PH':'pH', 'WTEMP' : 'Water Temperature', 'DO': 'Dissolved Oxygen', 'DO_SAT_P': 'Dissolved oxygen relative to theoretical value at saturation (%)', 'SALINITY': 'Salinity in-situ measured with probe', 'SPCOND' :'Specific Conductance At 25 C'}

In [470]:
water_clean = water_clean.drop(columns=['ReportedUnits','WQMethod'])

# Combining datasets

There are two options here: 
- We can turn the parameters into columns. This way, each parameter has a column with the measured value. We can then combine rows based on the `FieldActivityId`, so that all measurements taking at the same time and location (including depth) are in the same row.
- The other option is to rename the parameter and measured value columns for each of the datasets for a consistent naming scheme. Since the different datasets have different columns, we will also need to drop any excess columns.

## Functions for analyzing column names

First, let's see what columns are common to all for cleaned dataframes and which are unique. Since we will want to run this step a few times, we will define a function.

In [471]:
# define dictionary that stores the variable names
def dictionary_variable_names(*dfs):
    # create empty dictionary
    variable_names = {}
    
    # iterate over the dataframes
    for df in dfs:
        # get the variable name
        var_name = [var for var, val in globals().items() if val is df][0]
        
        # add the variable name to the dictionary
        variable_names[var_name] = df
    
    return variable_names

In [472]:
def analyze_columns(*dfs):
    df_dict = dictionary_variable_names(*dfs)
    # Extract column sets
    columns_sets = {name: set(df.columns) for name, df in df_dict.items()}
    
    # Find common columns
    common_columns = set.intersection(*columns_sets.values())
    
    # Find unique columns for each DataFrame
    unique_columns = {
        name: columns - set.union(*(other_columns for other_name, other_columns in columns_sets.items() if other_name != name))
        for name, columns in columns_sets.items()
    }
    
    # Display results
    print("Common Columns:")
    print(common_columns)
    
    for name, df_columns in columns_sets.items():
        print(f"\nUnique Columns in {name}:")
        print(unique_columns[name])
        print(f"Columns in {name} not in common_columns:")
        print(df_columns - common_columns)


## Turn parameters into columns

### More cleaning

In [473]:
def parameter_to_columns(dataframe,columns_to_group):
    # Reset index to use row numbers as the index
    df_reset = dataframe.reset_index(drop=True)

    # Pivot the DataFrame while preserving non-pivoted columns
    df_pivoted = df_reset.pivot_table(index=df_reset.index, columns='ReportedParameter', values='ReportedValue', aggfunc='first')

    # Combine pivoted result with the original DataFrame columns not involved in the pivot
    df_pivoted = df_reset.drop(columns=['ReportedParameter','ReportedValue']).join(df_pivoted)

    #check unique combinations
    #this allows us to check that we havent lost data
    unique_combinations = df_pivoted[columns_to_group].drop_duplicates()

    # Check we haven't lost unique non-empty values
    for col in columns_to_group:
        # Filter out empty values
        df_combined_nonempty = df_pivoted[col].dropna()
        df_pivoted_nonempty = dataframe[col].dropna()
        
        # Check if the number of unique non-empty values matches
        unique_check = df_combined_nonempty.unique().size == df_pivoted_nonempty.unique().size
        
        # Print the results
        print("Checking unique values in " ,col, unique_check)
    
    # Create a copy of the DataFrame for processing
    df_processed = df_pivoted.copy()

    # Create a unique identifier for each group based on the columns to match
    df_processed['UniqueID'] = df_processed[columns_to_group].astype(str).agg('-'.join, axis=1)

    # Group by the unique identifier
    df_combined = df_processed.groupby('UniqueID', as_index=False).first()

    # Drop the UniqueID column and remove duplicates
    df_really_clean = df_combined.drop(columns='UniqueID').drop_duplicates()

    return df_really_clean

The columns to group come from the monitoring data, so should be present in all datasets. We will double check this with the `analyze_columns` function from the [functions for analyzing column names](#functions-for-analyzing-column-names) subsection.

In [474]:
analyze_columns(sediment_clean, biomass_clean, taxonomic_clean,water_clean)

Common Columns:
{'PDepth', 'Salzone', 'CBSeg2003Description', 'FieldActivityId', 'CBSeg2003', 'SampleTime', 'Latitude', 'Longitude', 'SampleDate', 'TotalDepth', 'ProjectIdentifier', 'Source', 'Station', 'SampleVolume'}

Unique Columns in sediment_clean:
set()
Columns in sediment_clean not in common_columns:
{'ReportedParameter', 'ReportedValue', 'SampleReplicate', 'EventId'}

Unique Columns in biomass_clean:
{'BiologicalEventId'}
Columns in biomass_clean not in common_columns:
{'ReportedParameter', 'ReportedValue', 'BiologicalEventId', 'SampleReplicate'}

Unique Columns in taxonomic_clean:
{'SampleType', 'LatinName', 'ReportingValue'}
Columns in taxonomic_clean not in common_columns:
{'SampleType', 'ReportingValue', 'LatinName', 'EventId'}

Unique Columns in water_clean:
{'SampleDepth'}
Columns in water_clean not in common_columns:
{'ReportedParameter', 'ReportedValue', 'SampleDepth', 'EventId', 'SampleReplicate'}


In [475]:
columns_to_group = ['PDepth', 'Salzone', 'CBSeg2003Description', 'FieldActivityId', 'CBSeg2003', 'SampleTime', 'Latitude', 'Longitude', 'SampleDate', 'TotalDepth', 'ProjectIdentifier', 'Source', 'Station', 'SampleVolume']

Let's apply the function!

In [476]:
sediment_really_clean = parameter_to_columns(sediment_clean,columns_to_group)

Checking unique values in  PDepth True
Checking unique values in  Salzone True
Checking unique values in  CBSeg2003Description True
Checking unique values in  FieldActivityId True
Checking unique values in  CBSeg2003 True
Checking unique values in  SampleTime True
Checking unique values in  Latitude True
Checking unique values in  Longitude True
Checking unique values in  SampleDate True
Checking unique values in  TotalDepth True
Checking unique values in  ProjectIdentifier True
Checking unique values in  Source True
Checking unique values in  Station True
Checking unique values in  SampleVolume True


In [477]:
biomass_really_clean = parameter_to_columns(biomass_clean,columns_to_group)

Checking unique values in  PDepth True
Checking unique values in  Salzone True
Checking unique values in  CBSeg2003Description True
Checking unique values in  FieldActivityId True
Checking unique values in  CBSeg2003 True
Checking unique values in  SampleTime True
Checking unique values in  Latitude True
Checking unique values in  Longitude True
Checking unique values in  SampleDate True
Checking unique values in  TotalDepth True
Checking unique values in  ProjectIdentifier True
Checking unique values in  Source True
Checking unique values in  Station True
Checking unique values in  SampleVolume True


Some final cleaning for Taxonomic

In [478]:
taxonomic_clean = taxonomic_clean.rename(columns={'ReportingValue': 'ReportedValue','LatinName':'ReportedParameter'})

In [479]:
taxonomic_really_clean = parameter_to_columns(taxonomic_clean,columns_to_group)

Checking unique values in  PDepth True
Checking unique values in  Salzone True
Checking unique values in  CBSeg2003Description True
Checking unique values in  FieldActivityId True
Checking unique values in  CBSeg2003 True
Checking unique values in  SampleTime True
Checking unique values in  Latitude True
Checking unique values in  Longitude True
Checking unique values in  SampleDate True
Checking unique values in  TotalDepth True
Checking unique values in  ProjectIdentifier True
Checking unique values in  Source True
Checking unique values in  Station True
Checking unique values in  SampleVolume True


In [480]:
water_really_clean = parameter_to_columns(water_clean,columns_to_group)

Checking unique values in  PDepth True
Checking unique values in  Salzone True
Checking unique values in  CBSeg2003Description True
Checking unique values in  FieldActivityId True
Checking unique values in  CBSeg2003 True
Checking unique values in  SampleTime True
Checking unique values in  Latitude True
Checking unique values in  Longitude True
Checking unique values in  SampleDate True
Checking unique values in  TotalDepth True
Checking unique values in  ProjectIdentifier True
Checking unique values in  Source True
Checking unique values in  Station True
Checking unique values in  SampleVolume True


Let's get rid of some extra columns before merging. `BiologicalEventId` is only in BioMass, `EventId` is simply a different system for recoding the same information as `FieldActivityId`. `SampleReplicate` should not matter, but migh prevent some merging.

In [481]:
biomass_really_clean = biomass_really_clean.drop(columns=['SampleReplicate','BiologicalEventId'])
taxonomic_really_clean = taxonomic_really_clean.drop(columns=['EventId'])
water_really_clean = water_really_clean.drop(columns=['EventId','SampleReplicate'])

### Combining Dataset

Now we combine on 'CBSeg2003', 'CBSeg2003Description', 'Station', 'Latitude', 'Longitude',
       'EventId', 'Source', 'SampleDate', 'SampleDepth'

In [482]:
# Merge sediment_really_clean and biomass_really_clean
merged_df = pd.merge(sediment_really_clean, biomass_really_clean, how='outer', on=[col for col in sediment_really_clean.columns if col in biomass_really_clean.columns], suffixes=('', '_biomass_really_clean'))

# Merge the result with taxonomic_really_clean
merged_df = pd.merge(merged_df, taxonomic_really_clean, how='outer', on=[col for col in merged_df.columns if col in taxonomic_really_clean.columns and not col.endswith('_biomass_really_clean')], suffixes=('', '_taxonomic_really_clean'))

# Merge the result with water_really_clean
merged_df = pd.merge(merged_df, water_really_clean, how='outer', on=[col for col in merged_df.columns if col in water_really_clean.columns and not col.endswith(('_biomass_really_clean', '_taxonomic_really_clean'))], suffixes=('', '_water_really_clean'))

# Reset the index for better readability
merged_df = merged_df.reset_index(drop=True)

There are also some issues with different precisions for latitude and longitude causing lack of matching.

In [483]:
#  Define columns for matching
match_columns = [
    'CBSeg2003', 'CBSeg2003Description', 'Station', 'FieldActivityId', 'SampleDate',
    'SampleTime', 'TotalDepth', 'Source', 'ProjectIdentifier'
]

# Generate a composite key based on the matching columns
merged_df['unique_key'] = merged_df[match_columns].apply(lambda row: tuple(row.fillna('missing')), axis=1)

# Handle Latitude and Longitude with precision
# Keep the most precise value for Latitude and Longitude
merged_df['Latitude'] = merged_df.groupby('unique_key')['Latitude'].transform(lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan)
merged_df['Longitude'] = merged_df.groupby('unique_key')['Longitude'].transform(lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan)

# Aggregate the groups
result_df = merged_df.groupby('unique_key').first().reset_index()

# Drop the unique_key column from the result
result_df = result_df.drop(columns=['unique_key'], errors='ignore')


Also, let's strip whitespace, which should have been done before.

In [484]:
for col in result_df.columns:
    if result_df[col].dtype == 'object':
        result_df[col] = result_df[col].str.strip()

In [485]:
result_df.to_csv('../data/plank_ChesapeakeBayBenthic_clean_wide.csv')

## Renaming columns instead of paramenters into columns

Let's make sure all of the datasets have the same columns

In [487]:
analyze_columns(sediment_clean,biomass_clean,taxonomic_clean,water_clean)

Common Columns:
{'PDepth', 'ReportedParameter', 'Salzone', 'CBSeg2003Description', 'FieldActivityId', 'CBSeg2003', 'SampleTime', 'ReportedValue', 'Latitude', 'Longitude', 'SampleDate', 'TotalDepth', 'ProjectIdentifier', 'Source', 'Station', 'SampleVolume'}

Unique Columns in sediment_clean:
set()
Columns in sediment_clean not in common_columns:
{'SampleReplicate', 'EventId'}

Unique Columns in biomass_clean:
{'BiologicalEventId'}
Columns in biomass_clean not in common_columns:
{'BiologicalEventId', 'SampleReplicate'}

Unique Columns in taxonomic_clean:
{'SampleType'}
Columns in taxonomic_clean not in common_columns:
{'SampleType', 'EventId'}

Unique Columns in water_clean:
{'SampleDepth'}
Columns in water_clean not in common_columns:
{'SampleReplicate', 'SampleDepth', 'EventId'}


Since `EventId` encodes the same information as `FieldActivityId`, but no used by every dataset, we will drop it. Similar with `BiologicalEventId`. `SampleReplicate` is another column that would be helpful to double check if we were combining rows, but is not present in every dataset, so we will drop it. Let

In [488]:
sediment_really_clean = sediment_clean.drop(columns=['EventId','SampleReplicate'])
biomass_really_clean = biomass_clean.drop(columns=['BiologicalEventId','SampleReplicate'])
taxonomic_really_clean = taxonomic_clean.drop(columns=['EventId','SampleType'])
water_really_clean = water_clean.drop(columns=['EventId','SampleDepth','SampleReplicate'])

In [489]:
analyze_columns(sediment_really_clean,biomass_really_clean,taxonomic_really_clean,water_really_clean)

Common Columns:
{'PDepth', 'ReportedParameter', 'Salzone', 'CBSeg2003Description', 'FieldActivityId', 'CBSeg2003', 'SampleTime', 'ReportedValue', 'Latitude', 'Longitude', 'SampleDate', 'TotalDepth', 'ProjectIdentifier', 'Source', 'Station', 'SampleVolume'}

Unique Columns in sediment_really_clean:
set()
Columns in sediment_really_clean not in common_columns:
set()

Unique Columns in biomass_really_clean:
set()
Columns in biomass_really_clean not in common_columns:
set()

Unique Columns in taxonomic_really_clean:
set()
Columns in taxonomic_really_clean not in common_columns:
set()

Unique Columns in water_really_clean:
set()
Columns in water_really_clean not in common_columns:
set()


Now we need to use a consistent naming convention. Let's use  `ReportedParameter` and `ReportedValue`.

In [490]:
taxonomic_really_clean = taxonomic_clean.rename(columns={'LatinName':'ReportedParameter','ReportingValue':'ReportedValue'})


In [491]:
analyze_columns(sediment_really_clean,biomass_really_clean,taxonomic_really_clean,water_really_clean)

Common Columns:
{'PDepth', 'ReportedParameter', 'Salzone', 'CBSeg2003Description', 'FieldActivityId', 'CBSeg2003', 'SampleTime', 'ReportedValue', 'Latitude', 'Longitude', 'SampleDate', 'TotalDepth', 'ProjectIdentifier', 'Source', 'Station', 'SampleVolume'}

Unique Columns in sediment_really_clean:
set()
Columns in sediment_really_clean not in common_columns:
set()

Unique Columns in biomass_really_clean:
set()
Columns in biomass_really_clean not in common_columns:
set()

Unique Columns in taxonomic_really_clean:
{'SampleType', 'EventId'}
Columns in taxonomic_really_clean not in common_columns:
{'SampleType', 'EventId'}

Unique Columns in water_really_clean:
set()
Columns in water_really_clean not in common_columns:
set()


Now we can concatinate the datasets.

In [493]:
combined_df = pd.concat([sediment_really_clean,biomass_really_clean,taxonomic_really_clean,water_really_clean], ignore_index=True)

for col in combined_df.columns:
    if combined_df[col].dtype == 'object':
        combined_df[col] = combined_df[col].str.strip()

For some reason 'EventId','SampleType' did not drop, so let's do that again

In [495]:
combined_df = combined_df.drop(columns=['EventId','SampleType'])

In [496]:
combined_df.to_csv('../data/plank_ChesapeakeBayBenthic_clean_tall.csv')